In [1]:
import pandas as pd
import re
import json
import xml.etree.ElementTree as xet
import requests as req
import time
import glob
import csv
import regex
import tqdm
import pytz
import numpy as np
import io
from bs4 import BeautifulSoup

In [2]:
def parse_sitemap(fn):

    xml = xet.parse(fn)
    xml_root = xml.getroot()
    news_list = []
    
    for news in xml_root:
        try:
            result = {}
            result['url'] = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}loc').text
            lastmod = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}lastmod')
            if lastmod is not None:
                result['lastmod'] = lastmod.text

            #title = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:title').text
            #keywords = news.find('{http://www.sitemaps.org/schemas/sitemap/0.9}news:keywords').text
            #news_list.append(dict(zip(['url', 'lastmod'], [url, lastmod])))
            news_list.append(result)
        except:
            continue

    return news_list

In [4]:
sitemaps = parse_sitemap(io.StringIO(req.get('https://svpressa.ru/sitemap.xml').content.decode()))
sitemaps_filtered = []
for sitemap in sitemaps:
    if re.search(r'2020|2019', sitemap['url']):
        sitemaps_filtered.append(sitemap)

In [5]:
news_db = []

for fn in tqdm.tqdm_notebook(sitemaps_filtered):
    s1 = time.time()

    try:
        text = io.StringIO(req.get(fn['url']).content.decode())
        news = parse_sitemap(text)
    except Exception as e:
        print(e)
        continue
        
    news_db += news
    s2 = time.time()
    time.sleep(max(1 - s2 + s1, 0))

<ipython-input-5-73c316e576b5>:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for fn in tqdm.tqdm_notebook(sitemaps_filtered):


In [6]:
df = pd.DataFrame(news_db)
df['lastmod'] = pd.to_datetime(df['lastmod'], format='%Y-%m-%dT%H:%M:%S', utc=True)
df.head()

,url,lastmod
0,https://svpressa.ru/accidents/news/220858/,2019-01-01 12:42:07+00:00
1,https://svpressa.ru/sport/news/220860/,2019-01-01 13:42:57+00:00
2,https://svpressa.ru/accidents/news/220855/,2019-01-01 16:20:20+00:00
3,https://svpressa.ru/accidents/news/220853/,2019-01-01 16:20:43+00:00
4,https://svpressa.ru/sport/news/220852/,2019-01-01 16:21:11+00:00


In [7]:
df[(df.lastmod >= '2019-11-01')].to_json('news_svpressa.json', date_unit='s', date_format='iso')
df[(df.lastmod >= '2019-11-01')].shape

(35497, 2)

# Parsing

In [14]:
def parse_news(text):
    soup = BeautifulSoup(text, features='lxml')
    
    news_text = ''
    sub = soup.find('h2', attrs={'class': 'b-text__subtitle'})
    if sub:
        news_text = sub.get_text(" ", strip=True)

    news_body = soup.find(name='div', attrs={'class': 'b-text__content'})
    if news_body:
        news_ps = news_body.find_all(name='p')
        if news_ps:
            news_text += " " + " ".join([news_p.get_text(" ", strip=True) for news_p in news_ps])
        news_text = re.sub(r'[\n\r]+', ' ', news_text).replace(u'\xa0', u' ')
        
    tags_elem = soup.find_all(name="a", attrs={'class': "b-tag__link"})
    tags = re.sub(r'[\n\r]+', ' ', ",".join([tag.get_text("content") for tag in tags_elem])).replace('#', '') if tags_elem else ''
    
    description_elem = soup.find(name="meta", attrs={'name': "description"})
    description = re.sub(r'[\n\r]+|<[^>]+>', ' ', description_elem.get('content')) if description_elem else ''
        
    return {'news_text': news_text, 'tags': tags, 'description': description}

In [15]:
parse_news(req.get('https://svpressa.ru/society/article/235150/').content)

{'news_text': 'Власть так скоропалительно решала «проблему 2022», повышая возраст выхода на пенсию, что забыла о дефиците ПФР Из-за теневой занятости граждан социальные фонды недополучают около 2,3 трлн. рублей. Об этом заявила вице-премьер Татьяна Голикова в интервью «Известиям». «По некоторым, в том числе косвенным, данным, в неформальном секторе трудится порядка 15 млн. человек. Что касается превышения расходов над доходами граждан, по данным за 2017 год (данных за 2018-й еще нет), разница составляет 13,3 трлн. рублей — это 14,5% ВВП. Часть из них — это недоуплаченные налоги. Если говорить о страховых взносах, то это приблизительно 2,3 трлн. рублей», — пояснила Голикова. Вице-премьер добавила, что для вывода из тени «серых» доходов с 1 января реализуется пилотный проект, предлагающий регистрацию самозанятым. По последним данным, зарегистрировались около 102 тыс. человек — это почти 7,6 млрд. рублей оборота. Заметим, сам теневой сектор от этого меньше не становится. В марте Росстат п